In [47]:
import time
import torch
from torch import nn, optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5),# in_channels, out_channels, keneral_size
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2),# keneral_size, stride
            nn.Conv2d(6, 16, 5),
            nn.Sigmoid(),
            nn.MaxPool2d(2, 2)
            )
        self.fc = nn.Sequential(
            nn.Linear(16 * 4 * 4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
            )
        
    def forward(self, image):
        feature = self.conv(image)
        output = self.fc(feature.view(image.shape[0],-1))
        return output
        

In [45]:
net = LeNet()
net

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [46]:
import torchvision
batch_size = 256
data_train=torchvision.datasets.FashionMNIST(root='D:/Datasets', 
                                              train=True, 
                                              download=True,
                                              transform=torchvision.transforms.ToTensor())
data_test=torchvision.datasets.FashionMNIST(root='D:/Datasets', 
                                              train=False, 
                                              download=True,
                                              transform=torchvision.transforms.ToTensor())
train_iter = torch.utils.data.DataLoader(data_train,
                                        shuffle=True,
                                        batch_size=batch_size)
test_iter = torch.utils.data.DataLoader(data_test,
                                        shuffle=True,
                                        batch_size=batch_size)

In [41]:
def evaluate_accuracy(data_iter, net, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device)).argmax(dim=1) == 
                             y.to(device)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # ⾃定义的模型, 3.13节之后不会⽤到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames):
                # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X,is_training=False).argmax(dim=1) ==
                                 y).float().sum().item()
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

In [42]:
def train_ch5(net, train_iter, test_iter, batch_size, optimizer,
device, num_epochs):
    net = net.to(device)
    print("training on ", device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0,time.time()
        for X, y in train_iter:
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) ==y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1
        test_acc = evaluate_accuracy(test_iter, net)
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f,time %.1f sec' 
              % (epoch + 1, train_l_sum / batch_count,
                train_acc_sum / n, test_acc, time.time() - start))

In [43]:
lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cpu
epoch 1, loss 1.8229, train acc 0.334, test acc 0.578,time 29.1 sec
epoch 2, loss 0.4680, train acc 0.641, test acc 0.681,time 26.1 sec
epoch 3, loss 0.2534, train acc 0.720, test acc 0.726,time 28.1 sec
epoch 4, loss 0.1684, train acc 0.746, test acc 0.745,time 27.6 sec
epoch 5, loss 0.1232, train acc 0.764, test acc 0.764,time 29.2 sec
